# Introduction

The purpose of this notebook is to gather the data necessary to import into the [Analysis Notebook.ipynb](https://github.com/bentson1187/Stock-Market-Inefficiency-Analysis/blob/9b9f50faf46bb718a2e2c40d85d93eb37354fe2c/Analysis%20Notebook.ipynb) file that is in this github repo and to feed the accompanying [Tableau dashboard](https://public.tableau.com/app/profile/brian8863/viz/CapstoneDashboard_16252496264850/HoldingsDashboard?publish=yes). I will be grabbing data from the [FMP API](https://financialmodelingprep.com/developer/docs) and creating .csv files that feed both the analysis and the dashboard. 

# Import Packages

In [4]:
#data collection
import requests
import json
from urllib.request import urlopen

import pandas as pd
import seaborn as sns

pd.options.display.float_format = '{:,}'.format

# Global Functions

In [ ]:
#get data from api
def get_api_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

In [1]:
def get_symbols_data(api_key):
  '''
  This function gathers all the stock symbols in the FMP API

  Keyword arguments:
  api_key -- A users api key
  '''
  
  symbols_url = 'https://financialmodelingprep.com/api/v3/financial-statement-symbol-lists?apikey='+api_key

  symbols_list = get_api_data(symbols_url)

  return symbols_list


In [ ]:
def get_value_metrics(api_key, symbols_list, num_symbols):
  '''
  This function gets key metrics from the FMP API
  
  Keyword arguments:
  api_key -- A users api key
  symbols_list -- a list of stock symbols
  num_symbols -- the number of symbols to grab from the API out of the total 
  number in the symbols_list
  '''
  results = {}
  for stock in symbols_list[:num_symbols]:
    try:
      
      result = get_api_data(f"https://financialmodelingprep.com/api/v3/key-metrics-ttm/{stock}?limit=40&apikey={api_key}")[0]
      results[stock] = pd.Series(result)
      
    
    except:
      pass

  df = pd.DataFrame(results).T


  #create list of columns to keep
  keep_cols = ['revenuePerShareTTM','peRatioTTM','marketCapTTM',
               'priceToSalesRatioTTM','enterpriseValueOverEBITDATTM',
               'evToFreeCashFlowTTM','evToOperatingCashFlowTTM']
  df = df[keep_cols]

  df.reset_index(inplace=True)
  df.rename(columns={'index':'symbol'}, inplace=True)

  return df

In [ ]:
def get_price_metrics(api_key, symbols_list, num_symbols, timeframe):
  '''
  This function gets stock prices from the FMP API and then calculates metrics
  based on those stock prices.
  
  Keyword arguments:
  api_key -- A users api key
  symbols_list -- a list of stock symbols
  num_symbols -- the number of symbols to grab from the API out of the total 
  number in the symbols_list
  timeframe -- the length of time in days to get the stock prices for
  '''
  df_list = []
  
  for stock in symbols_list[:num_symbols]:
    try:
      response = get_api_data(f"https://financialmodelingprep.com/api/v3/historical-price-full/{stock}?timeseries={timeframe}&apikey={api_key}")['historical']
      df = pd.DataFrame(response)
      df['symbol'] = stock
      df['percent_change'] = (df.loc[0,'close']  - df.loc[len(df)-1, 'open'])/df.loc[len(df)-1, 'open']
      df['percent_negative'] = df['changePercent'] < 0
      df['percent_positive'] = df['changePercent'] > 0
      df['percent_flat'] = df['changePercent'] == 0
      df['total_percent_negative'] = df['percent_negative'].sum()/len(df)
      df['total_percent_positive'] = df['percent_positive'].sum()/len(df)

      df.loc[df['percent_change'] < 0, 'FIP'] = -1 * (df['total_percent_negative'] - df['total_percent_positive'])
      df.loc[df['percent_change'] > 0, 'FIP'] = 1 * (df['total_percent_negative'] - df['total_percent_positive'])

      df = df.loc[[0]]
      df_list.append(df)

    except:
      pass
      
  df = pd.concat(df_list)

  return df

In [ ]:
def get_company_profile(api_key, symbols_list, num_symbols):
  '''
  This function gets company profile information from the FMP API
  
  Keyword arguments:
  api_key -- A users api key
  symbols_list -- a list of stock symbols
  num_symbols -- the number of symbols to grab from the API out of the total 
  '''
  
  df_list = []
  for stock in symbols_list[:num_symbols]:
    try:
      results = get_api_data(f"https://financialmodelingprep.com/api/v3/profile/{stock}?apikey={api_key}")[0]
      df = pd.DataFrame.from_dict(results, orient='index').T
      df_list.append(df)
  
    except:
      pass

  df = pd.concat(df_list)

  return df

In [ ]:
def get_rd_spend(api_key, symbols_list, num_symbols):
  '''
  This function gets research and development spend from the FMP API
  
  Keyword arguments:
  api_key -- A users api key
  symbols_list -- a list of stock symbols
  num_symbols -- the number of symbols to grab from the API out of the total 
  '''
  
  results = {}
  
  for stock in symbols_list[:num_symbols]:
    try:
      result = get_api_data(f"https://financialmodelingprep.com/api/v3/income-statement/{stock}?limit=1&apikey={api_key}")[0]['researchAndDevelopmentExpenses']

      results[stock] = result

    except:
      pass
      
  df = pd.DataFrame(results.values(), columns=['rd_spend'], index=results.keys())
  df.reset_index(inplace=True)
  df.rename(columns={'index':'symbol'}, inplace=True)

  return df

In [ ]:
def get_enterprise_value(api_key, symbols_list, num_symbols):
  '''
  This function gets the enterprise value from the FMP API
  
  Keyword arguments:
  api_key -- A users api key
  symbols_list -- a list of stock symbols
  num_symbols -- the number of symbols to grab from the API out of the total 
  '''
  results = {}
  
  for stock in symbols_list[:num_symbols]:
    try:
      
      results[stock] = get_api_data(f'https://financialmodelingprep.com/api/v3/enterprise-values/{stock}?limit=40&apikey={api_key}')[0]['enterpriseValue']

    except:
      pass
    
  df = pd.DataFrame(results.values(), columns=['enterprise_value'], index=results.keys())
  df.reset_index(inplace=True)
  df.rename(columns={'index':'symbol'}, inplace=True)

  return df


In [ ]:
def get_income_statement(api_key, symbols_list, num_symbols):
  '''
  This function gets the income statement information from the FMP API
  
  Keyword arguments:
  api_key -- A users api key
  symbols_list -- a list of stock symbols
  num_symbols -- the number of symbols to grab from the API out of the total 
  '''
  df_list = []

  for stock in symbols_list[:num_symbols]:
    try:

      df_list.append(get_api_data(f'https://financialmodelingprep.com/api/v3/income-statement/{stock}?limit=120&apikey={api_key}')[:num_symbols])

    except:
      pass

  df = pd.DataFrame(df_list)
  #df = df[['symbol','ebitda','revenue','netIncome']]
  df.rename(columns={'revenue':'sales', 'netIncome':'earnings'}, inplace=True)
  
  return df

In [ ]:
def get_quarterly_income_statement_timeseries(api_key, symbols_list, num_symbols, num_quarters):
  '''
  This function gets the quarterly income statements from the FMP API
  
  Keyword arguments:
  api_key -- A users api key
  symbols_list -- a list of stock symbols
  num_symbols -- the number of symbols to grab from the API out of the total 
  num_quarters -- the number of quarters to grab the income statements from
  '''
  df_list = []

  for stock in symbols_list[:num_symbols]:
    try:

      df_list.append(pd.DataFrame(get_api_data(f'https://financialmodelingprep.com/api/v3/income-statement/{stock}?period=quarter&limit=400&apikey={api_key}')[:num_quarters]))

    except:
      pass

  df = pd.concat(df_list)
  df.rename(columns={'revenue':'sales', 'netIncome':'earnings'}, inplace=True)
  df.reset_index(inplace=True)
  df.drop(columns=['index'], inplace=True)
  
  return df

In [ ]:
def get_quarterly_cashflow_statement_timeseries(api_key, symbols_list, num_symbols, num_quarters):
  '''
  This function gets the quarterly cashflow statements from the FMP API
  
  Keyword arguments:
  api_key -- A users api key
  symbols_list -- a list of stock symbols
  num_symbols -- the number of symbols to grab from the API out of the total 
  num_quarters -- the number of quarters to grab the income statements from
  '''
  df_list = []

  for stock in symbols_list[:num_symbols]:
    try:

      df_list.append(pd.DataFrame(get_api_data(f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{stock}?apikey={api_key}&period=quarter&limit=400')[:num_quarters]))

    except:
      pass

  df = pd.concat(df_list)
  df.reset_index(inplace=True)
  df.drop(columns=['index'], inplace=True)
  
  return df

In [ ]:
def get_quarterly_key_metrics_statement_timeseries(api_key, symbols_list, num_symbols, num_quarters):
  '''
  This function gets the quarterly key metrics statements from the FMP API
  
  Keyword arguments:
  api_key -- A users api key
  symbols_list -- a list of stock symbols
  num_symbols -- the number of symbols to grab from the API out of the total 
  num_quarters -- the number of quarters to grab the income statements from
  '''
  df_list = []

  for stock in symbols_list[:num_symbols]:
    try:

      df_list.append(pd.DataFrame(get_api_data(f"https://financialmodelingprep.com/api/v3/key-metrics/{stock}?period=quarter&limit={num_quarters}&apikey={api_key}")))

    except:
      pass

  df = pd.concat(df_list)
  df.reset_index(inplace=True)
  df.drop(columns=['index'], inplace=True)
  
  return df

In [ ]:
def get_stock_insider_trading(api_key, symbols_list, num_symbols, num_records):
  '''
  This function gets the stock insider trading information from the FMP API
  
  Keyword arguments:
  api_key -- A users api key
  symbols_list -- a list of stock symbols
  num_symbols -- the number of symbols to grab from the API out of the total 
  num_records -- the number of records to get per stock
  '''
  df_list = []

  for stock in symbols_list[:num_symbols]:
    try:

      df_list.append(pd.DataFrame(get_api_data(f"https://financialmodelingprep.com/api/v4/insider-trading?symbol={stock}&limit={num_records}&apikey={api_key}")))

    except:
      pass

  df = pd.concat(df_list)
  df.reset_index(inplace=True)
  df.drop(columns=['index'], inplace=True)
  
  return df

In [ ]:
def get_cashflows(api_key, symbols_list, num_symbols):
  '''
  This function gets the latest cashflow statement from the FMP API
  
  Keyword arguments:
  api_key -- A users api key
  symbols_list -- a list of stock symbols
  num_symbols -- the number of symbols to grab from the API out of the total 
  '''
  df_list = []

  for stock in symbols_list[:num_symbols]:
    try:

      df_list.append(get_api_data(f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{stock}?apikey={api_key}&limit=190')[0])

    except:
      pass

  df = pd.DataFrame(df_list)
  df = df[['symbol','freeCashFlow','operatingCashFlow']]

  return df

In [ ]:
def get_daily_price(api_key, symbols_list, num_symbols, timeframe):
  '''
  This function gets the the daily stock price from the FMP API
  
  Keyword arguments:
  api_key -- A users api key
  symbols_list -- a list of stock symbols
  num_symbols -- the number of symbols to grab from the API out of the total 
  timeframe -- the number of days to get the stock price for
  '''  
  df_list = []
  
  for stock in symbols_list[:num_symbols]:
    try:
      response = get_api_data(f"https://financialmodelingprep.com/api/v3/historical-price-full/{stock}?timeseries={timeframe}&apikey={api_key}")['historical']
      df = pd.DataFrame(response)
      df['symbol'] = stock
      df_list.append(df)

    except:
      pass
      
  df = pd.concat(df_list)

  return df

In [ ]:
def get_stock_news(api_key, symbols_list, num_symbols, num_stories):
  '''
  This function gets the stock news from the FMP API
  
  Keyword arguments:
  api_key -- A users api key
  symbols_list -- a list of stock symbols
  num_symbols -- the number of symbols to grab from the API out of the total 
  num_stores -- the number of stories to grab for each stock
  '''
  df_list = []

  for stock in symbols_list[:num_symbols]:
    try:

      df_list.append(pd.DataFrame(get_api_data(f'https://financialmodelingprep.com/api/v3/stock_news?tickers={stock}&limit={num_stories}&apikey={api_key}')))

    except:
      pass

  df = pd.concat(df_list)
  df.reset_index(inplace=True)
  df.drop(columns=['index'], inplace=True)
  
  return df

In [ ]:
def get_all_api_data(api_key, num_symbols, timeframe):
  '''
  This function utilizes previous functions to gather a single cohesive dataset
  for analysis and modeling
  
  Keyword arguments:
  api_key -- A users api key
  symbols_list -- a list of stock symbols
  num_symbols -- the number of symbols to grab from the API out of the total 
  timeframe -- the number of days to get the stock price for
  '''
  #get symbols list
  raw_symbols_list = get_symbols_data(api_key)

  #get key metrics ttm data from api
  df_value_metrics = get_value_metrics(api_key, raw_symbols_list, num_symbols)
  
  #standardize feature names
  df_value_metrics.rename(columns={'companyName':'company_name',
                                   'exchangeShortName':'exchange_short_name',
                                   'fullTimeEmployees':'full_time_employees',
                                   'isEtf':'is_etf','price_to_sales_ratio_ttm':'ps_ratio_ttm',
                                   'peRatioTTM':'pe_ratio_ttm',
                                   'evToOperatingCashFlowTTM':'ev_to_operating_cashflow_ttm',
                                   'enterpriseValueOverEBITDATTM':'ev_to_EBITDA_ttm',
                                   'evToFreeCashFlowTTM':'ev_to_free_cashflow_ttm',
                                   'priceToSalesRatioTTM':'ps_ratio_ttm',
                                   'revenuePerShareTTM':'revenue_per_share_ttm',
                                   'marketCapTTM':'market_cap_ttm', 'close':'stock_price'}
                          ,inplace=True)

  #define cols for nulls and negatives removal
  performance_metrics_cols = ['revenue_per_share_ttm','pe_ratio_ttm','market_cap_ttm',
                            'ps_ratio_ttm','ev_to_EBITDA_ttm',
                            'ev_to_free_cashflow_ttm', 'ev_to_operating_cashflow_ttm']

  #filter dataframe by removing a symbol which has a null in any of the performance metrics
  #create lists of symbol_ids
  pe_ratio_nulls = df_value_metrics.loc[df_value_metrics['pe_ratio_ttm'].isna() == True,'symbol'].tolist()
  ev_to_operating_cashflow_nulls = df_value_metrics.loc[df_value_metrics['ev_to_operating_cashflow_ttm'].isna() == True,'symbol'].tolist()
  ev_to_EBITDA_nulls = df_value_metrics.loc[df_value_metrics['ev_to_EBITDA_ttm'].isna() == True,'symbol'].tolist()
  ev_to_free_cashflow_nulls = df_value_metrics.loc[df_value_metrics['ev_to_free_cashflow_ttm'].isna() == True,'symbol'].tolist()
  price_to_sales_ratio_nulls = df_value_metrics.loc[df_value_metrics['ps_ratio_ttm'].isna() == True,'symbol'].tolist()
  market_cap_nulls = df_value_metrics.loc[df_value_metrics['market_cap_ttm'].isna() == True,'symbol'].tolist()

  #combine symbol_ids to a single list
  nulls_symbol_id_list = pe_ratio_nulls + ev_to_operating_cashflow_nulls + \
  ev_to_EBITDA_nulls + ev_to_free_cashflow_nulls + \
  price_to_sales_ratio_nulls + market_cap_nulls

  #create a set to remove duplicate symbol_ids
  nulls_symbol_id_list = set(nulls_symbol_id_list)

  #filter dataframe where the symbol_ids are not in the null symbol_ids list
  df_value_metrics = df_value_metrics.loc[~df_value_metrics['symbol'].isin(nulls_symbol_id_list)]

  #remove negative values for performance metrics
  for col in performance_metrics_cols:
    df_value_metrics.drop(df_value_metrics.loc[df_value_metrics[col] < 0].index, inplace=True)

  #create a clean list of symbols
  clean_symbols_list = list(df_value_metrics['symbol'].unique())

  #get stock price data from api
  df_price = get_price_metrics(api_key, clean_symbols_list, num_symbols=num_symbols, timeframe=timeframe)

  #merge dataframes
  df = pd.merge(df_value_metrics, df_price, on='symbol')

  #get company profile data from api
  df_company_profile = get_company_profile(api_key, clean_symbols_list, num_symbols=num_symbols)

  #merge dataframes
  df = pd.merge(df, df_company_profile, on='symbol')

  #get the r&d spend from API
  df_rd = get_rd_spend(api_key, clean_symbols_list, num_symbols=num_symbols)

  #merge dataframes
  df = pd.merge(df, df_rd, on='symbol')

  #get enterprise value from API
  df_ev = get_enterprise_value(api_key, clean_symbols_list, num_symbols)

  #merge dataframes
  df = pd.merge(df, df_ev, on='symbol')

  #get income statement from API
  df_income = get_income_statement(api_key, clean_symbols_list, num_symbols)

  #merge dataframes
  df = pd.merge(df, df_income, on='symbol')

  #get cashflows from API
  df_cashflows = get_cashflows(api_key, clean_symbols_list, num_symbols)

  #merge dataframes
  df = pd.merge(df, df_cashflows, on='symbol')

  df = df.round(2)

  return df

# Gather API Data

I will now use the API call functions to create .csv's for either modeling or for the Tableau dashboard.

In [ ]:
#retrieve api key
f = open('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/FMP_API_2.json')
api_key = json.load(f)['api_key']

In [ ]:
# #retrieve symbols list
# symbols_list = get_symbols_data(api_key)

## Pre-Modeling DataFrame

I will now gather the information needed for the analysis and modeling of the data.

In [ ]:
# #get all api data together for Modeling (1 hour 45 min)
# df_original = get_all_api_data(api_key, 25000, 253)

In [ ]:
# #write df to csv
# df_original.to_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Modeling/full dataset for capstone (7-12-21).csv',index=True)

In [ ]:
#import saved csv
df_original = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Modeling/full dataset for capstone (7-12-21).csv', 
                          index_col='Unnamed: 0')

In [ ]:
df_original.head()

,symbol,revenue_per_share_ttm,pe_ratio_ttm,market_cap_ttm,ps_ratio_ttm,ev_to_EBITDA_ttm,ev_to_free_cashflow_ttm,ev_to_operating_cashflow_ttm,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,percent_change,percent_negative,percent_positive,percent_flat,total_percent_negative,total_percent_positive,FIP,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,currency,cik,isin,cusip,exchange,exchangeShortName,industry,website,description,ceo,sector,country,fullTimeEmployees,phone,address,city,state,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,rd_spend,enterprise_value,ebitda,sales,earnings,freeCashFlow,operatingCashFlow
0,0003.HK,2.29,31.09,"219,653,328,020.16",5.41,26.61,89.15,25.72,2021-07-12,12.24,12.24,12.12,12.24,12.24,"14,263,608.0","14,263,608.0",0.0,0.0,12.2,"July 12, 21",0.0,0.05,False,False,True,0.47,0.41,0.07,12.38,0.440022,19558890,231009550336,0.34428600000000004,10.133333-13.047619,0.14,The Hong Kong and China Gas Company Limited,HKD,nan,HK0003000038,Y33370100,HKSE,HKSE,Utilities—Regulated Gas,http://www.towngas.com,"The Hong Kong and China Gas Company Limited, t...","Mr. Wing Kin Chan B.Sc., M.Sc., B.B.S., Hon.F...",Utilities,HK,"53,765.0",852 2963 3189,363 Java Road,North Point,NaN,NaN,nan,0.0,https://financialmodelingprep.com/image-stock/...,2020-12-15,False,False,True,0.0,"223,823,267,609.74","10,893,700,000.0","40,927,000,000.0","6,117,600,000.0","2,615,300,000.0","9,910,200,000.0"
1,0006.HK,0.63,14.43,"102,871,411,722.8",76.31,60.31,54.67,54.67,2021-07-12,47.55,47.7,47.2,47.4,47.4,"3,434,492.0","3,434,492.0",-0.15,-0.32,47.43,"July 12, 21",-0.0,0.07,True,False,False,0.46,0.46,0.0,48.25,0.553897,2705709,102978043904,2.81,39.2-50.55,0.85,Power Assets Holdings Limited,HKD,nan,HK0006000050,Y7092Q109,HKSE,HKSE,Utilities—Independent Power Producers,http://www.powerassets.com,"Power Assets Holdings Limited, an investment h...","Mr. Kin Ning Fok B.A., F.C.A., BA, DFM, FCA (...",Utilities,HK,13.0,852 2122 9122,Cheung Kong Center,Central,NaN,NaN,nan,0.0,https://financialmodelingprep.com/image-stock/...,2000-01-03,False,False,True,0.0,"86,358,004,175.94","1,120,000,000.0","1,270,000,000.0","6,132,000,000.0","960,000,000.0","962,000,000.0"
2,0012.HK,5.0,10.55,"179,359,050,000.0",7.42,26.33,38.7,38.7,2021-07-12,36.5,36.75,36.25,36.55,36.55,"1,774,792.0","1,774,792.0",0.05,0.14,36.52,"July 12, 21",0.0,0.18,False,True,False,0.49,0.45,0.04,37.0,0.978441,3803950,179116998656,1.8,27.2-38.85,0.45,Henderson Land Development Company Limited,HKD,nan,HK0012000102,Y31476107,HKSE,HKSE,Real Estate Services,http://www.hld.com,"Henderson Land Development Company Limited, an...","Mr. Ka-Shing Lee J.P., JP",Real Estate,HK,"9,065.0",852 2908 8888,Two International Finance Centre,Central,NaN,NaN,nan,0.0,https://financialmodelingprep.com/image-stock/...,2000-01-03,False,False,True,0.0,"235,071,704,841.0","12,957,000,000.0","25,020,000,000.0","10,192,000,000.0","7,072,000,000.0","7,072,000,000.0"
3,0016.HK,28.53,14.24,"337,237,895,708.4",4.08,10.52,9.38,8.8,2021-07-12,114.0,115.6,113.1,115.0,115.0,"3,408,483.0","3,408,483.0",1.0,0.88,114.57,"July 12, 21",0.01,0.13,False,True,False,0.46,0.49,-0.03,116.4,0.746821,3227665,337327194112,4.95,90.5-126.0,1.4,Sun Hung Kai Properties Limited,HKD,nan,HK0016000132,Y82594121,HKSE,HKSE,Real Estate—Development,http://www.shkp.com,Sun Hung Kai Properties Limited develops and i...,"Mr. Ping-Luen Kwok J.P., M.A., M.B.A., MA(Can...",Real Estate,HK,"39,500.0",852 2827 8111,Sun Hung Kai Centre,Wan Chai,NaN,NaN,nan,0.0,https://financialmodelingprep.com/image-stock/...,2020-12-15,False,False,True,0.0,"352,951,163,930.37","40,036,000,000.0","82,653,000,000.0","23,690,000,000.0","44,882,000,000.0","47,868,000,000.0"
4,0025.HK,21.14,5.51,"2,934,740,160.0",0.46,6.93,23.83,11.03,2021-07-12,9.7,9.7,9.7,9.7,9.7,"2,017.0","2,017.0",0.0,0.0,9.7,"July 12, 21",0.0,-0.06,False,False,True,0.17,0.21,0.04,9.8,0.302325,12634,2958894336,0.48,8.99-10.24,

## Post-Modeling DataFrame

I am now importing the data that is coming from the [Analysis Notebook.ipynb](https://github.com/bentson1187/Stock-Market-Inefficiency-Analysis/blob/9b9f50faf46bb718a2e2c40d85d93eb37354fe2c/Analysis%20Notebook.ipynb) file after analysis and modeling are complete. This is done here to provide a cleaned up list of stock symbols. There are over 20k stock symbols going into the analysis but only 4k afterwards. Getting additional information only on the cleaned set of stock symbols will create smaller .csv files and keep the Tableau dashboard performant.

In [ ]:
#import saved csv
df_eda = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Dashboarding/Exploration Data for Tableau EDA.csv')

In [ ]:
df_eda.head()

,symbol,revenue_per_share_ttm,pe_ratio_ttm,market_cap_ttm,ps_ratio_ttm,ev_to_EBITDA_ttm,ev_to_free_cashflow_ttm,ev_to_operating_cashflow_ttm,stock_price,percent_change,FIP,company_name,exchange_short_name,industry,sector,country,full_time_employees,state,rd_spend,enterprise_value,ebitda,sales,earnings,free_cashflow,operating_cashflow,owned,market_cap_group,value_ranking,action,anomaly,model_action,model_correct
0,IIVIP,"26,456.67",0.14,"25,296,557,880.0",0.01,0.07,0.1,0.07,298.21,0.49,0.11,II-VI Incorporated,NASDAQ,Scientific & Technical Instruments,Technology,US,"22,969.0",PA,"339,073,000.0","20,000,316,000.0","179,112,000.0","2,380,071,000.0","-67,029,000.0","156,665,000.0","297,292,000.0",no,Large-Cap,1,Risky Buy,-1,Risky Buy,True
1,MON,247.0,0.18,"301,486,776.7",0.04,0.12,0.32,0.18,9.7,-0.92,0.11,Monument Circle Acquisition Corp.,NYSE,Shell Companies,Basic Materials,US,-999.0,Unknown,"1,607,000,000.0","58,016,016,000.0","4,109,000,000.0","14,640,000,000.0","2,260,000,000.0","1,915,000,000.0","3,226,000,000.0",no,Small-Cap,2,Affordable,1,Affordable,True
2,1245.HK,23.45,1.66,"297,766,650.0",0.01,0.16,0.28,0.2,0.25,-0.11,0.15,"Niraku GC Holdings, Inc.",HKSE,Resorts & Casinos,"Hotels, Restaurants & Leisure",JP,"1,870.0",FUKUSHIMA-KEN,0.0,"-725,099,750.0","6,785,000,000.0","28,046,000,000.0","179,000,000.0","3,961,000,000.0","5,470,000,000.0",no,Micro-Cap,3,Affordable,1,Affordable,True
3,KGL.NS,1.65,0.81,"372,634,615.5",0.15,0.0,0.0,0.0,0.2,-0.2,-0.12,Karuturi Global Limited,NSE,Farm Products,Food Products,IN,-999.0,KARNATAKA,0.0,"2,353,258,800.0","784,779,000.0","2,295,912,000.0","519,307,000.0","925,061,000.0","925,061,000.0",no,Small-Cap,4,Affordable,1,Affordable,True
4,ALBDM.PA,25.99,0.85,"4,554,802.0",0.08,0.41,0.52,0.43,2.0,0.26,0.05,BD Multimedia SA,EURONEXT,Telecom Services,Communication Services,FR,16.0,ILE-DE-FRANCE,0.0,"610,714.0","-625,000.0","1,702,000.0","-364,000.0","-1,012,000.0","-873,000.0",no,Micro-Cap,5,Risky Buy,1,Risky Buy,True


## Tableau Dataframes

I will now utilize the API calls to create .csv files which are solely used for the [Tableau dashboard](https://public.tableau.com/app/profile/brian8863/viz/CapstoneDashboard_16252496264850/HoldingsDashboard?publish=yes).

In [ ]:
#get symbols list from modeling dataframe
clean_symbols_list = list(df_eda['symbol'].unique())

### 365 Day Stock Price

Gather 365 days of daily stock price data for Tableau

In [ ]:
# #get daily stock price (1 hr)
# df_stock_price = get_daily_price(api_key, clean_symbols_list, 6000, 253)

In [ ]:
# #write df to csv
# df_stock_price.to_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Dashboarding/365 Day Daily Stock Price.csv',index=True)

In [ ]:
#read df from csv
df_stock_price = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Dashboarding/365 Day Daily Stock Price.csv', 
                             index_col='Unnamed: 0')

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
df_stock_price.head()

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,symbol
0,2021-07-12,12.24,12.24,12.12,12.24,12.24,"14,165,155.0","14,165,155.0",0.0,0.0,12.2,"July 12, 21",0.0,0003.HK
1,2021-07-09,12.18,12.28,12.08,12.24,12.24,"33,125,720.0","33,125,720.0",0.06,0.493,12.2,"July 09, 21",0.0049299999999999995,0003.HK
2,2021-07-08,12.1,12.18,12.04,12.14,12.14,"15,727,871.0","15,727,871.0",0.04,0.331,12.12,"July 08, 21",0.00331,0003.HK
3,2021-07-07,12.08,12.1,12.0,12.06,12.06,"22,009,655.0","22,009,655.0",-0.02,-0.166,12.05333,"July 07, 21",-0.0016600000000000002,0003.HK
4,2021-07-06,12.18,12.2,12.04,12.12,12.12,"12,671,249.0","12,671,249.0",-0.06,-0.493,12.12,"July 06, 21",-0.0049299999999999995,0003.HK


### 6 Months Price Metrics

Get 6 months of stock prices and calculate the price metrics.

In [ ]:
# #get 6 months price metrics
# df_price_metrics = get_price_metrics(api_key, clean_symbols_list, 5000, 127)

In [ ]:
# #write df to csv
# df_price_metrics.to_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Dashboarding/6 Month Price Metrics.csv',index=True)

In [ ]:
#read df from csv
df_price_metrics = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Dashboarding/6 Month Price Metrics.csv', 
                             index_col='Unnamed: 0')

In [ ]:
df_price_metrics.head()

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,symbol,percent_change,percent_negative,percent_positive,percent_flat,total_percent_negative,total_percent_positive,FIP
0,2021-07-14,12.38,12.4,12.22,12.26,12.26,"12,400,361.0","12,400,361.0",-0.12,-0.9690000000000001,12.293330000000001,"July 14, 21",-0.00969,0003.HK,0.09093222650280144,True,False,False,0.4645669291338583,0.4094488188976378,0.055118110236220486
0,2021-07-14,48.75,49.0,48.65,48.8,48.8,"2,539,733.0","2,539,733.0",0.05,0.10300000000000001,48.81667,"July 14, 21",0.00103,0006.HK,0.17026376090494572,False,True,False,0.3779527559055118,0.5275590551181102,-0.14960629921259838
0,2021-07-14,37.0,37.05,36.2,36.25,36.25,"2,525,666.0","2,525,666.0",-0.75,-2.0269999999999997,36.5,"July 14, 21",-0.02027,0012.HK,0.1435330869547922,True,False,False,0.4330708661417323,0.5196850393700787,-0.08661417322834641
0,2021-07-14,116.6,116.8,114.4,114.9,114.9,"2,382,459.0","2,382,459.0",-1.7,-1.4580000000000002,115.36667,"July 14, 21",-0.014580000000000001,0016.HK,0.07584266640891388,True,False,False,0.4724409448818898,0.4960629921259842,-0.023622047244094502
0,2021-07-14,9.82,10.0,9.82,10.0,10.0,"8,023.0","8,023.0",0.18,1.8330000000000002,9.94,"July 14, 21",0.018330000000000003,0025.HK,0.02880658436213985,False,True,False,0.1889763779527559,0.2598425196850393,-0.07086614173228345


### Timeseries Quarterly Income Statement

Get quarterly income statements.

In [ ]:
# #get income statement df
# df_income_statement = get_quarterly_income_statement_timeseries(api_key, clean_symbols_list, 5000, 12)

In [ ]:
# #write df to csv
# df_income_statement.to_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Dashboarding/Quarterly Income Statement Timeseries.csv',index=True)

In [ ]:
#read from csv
df_income_statement = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Dashboarding/Quarterly Income Statement Timeseries.csv',
                                  index_col='Unnamed: 0')

In [ ]:
df_income_statement.head()

,date,symbol,reportedCurrency,fillingDate,acceptedDate,period,sales,costOfRevenue,grossProfit,grossProfitRatio,researchAndDevelopmentExpenses,generalAndAdministrativeExpenses,sellingAndMarketingExpenses,sellingGeneralAndAdministrativeExpenses,otherExpenses,operatingExpenses,costAndExpenses,interestExpense,depreciationAndAmortization,ebitda,ebitdaratio,operatingIncome,operatingIncomeRatio,totalOtherIncomeExpensesNet,incomeBeforeTax,incomeBeforeTaxRatio,incomeTaxExpense,earnings,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,link,finalLink
0,2019-09-30,0003.HK,HKD,2019-09-30,2019-09-30,Q3,"10,157,025,000.0","5,450,950,000.0","4,706,075,000.0",0.4633320288174933,"2,650,000.0",0.0,0.0,"2,996,375,000.0","-2,650,000.0","2,999,025,000.0","8,449,975,000.0","307,600,000.0","688,375,000.0","2,395,425,000.0",0.2358392344214965,"1,707,050,000.0",0.16806594450638845,0.0,"2,600,975,000.0",0.2560764593963291,"572,400,000.0","1,766,075,000.0",0.17387719337109045,0.09809525,0.09809525,"17,771,304,856.0","17,771,304,856.0",https://www1.hkexnews.hk/search/titlesearch.xhtml,https://www1.hkexnews.hk/search/titlesearch.xhtml
1,2019-06-30,0003.HK,HKD,2019-06-30,2019-06-30,Q2,"10,157,025,000.0","5,450,950,000.0","4,706,075,000.0",0.4633320288174933,"2,650,000.0",0.0,0.0,"2,996,375,000.0","-2,650,000.0","2,999,025,000.0","8,449,975,000.0","307,600,000.0","688,375,000.0","2,395,425,000.0",0.2358392344214965,"1,707,050,000.0",0.16806594450638845,0.0,"2,600,975,000.0",0.2560764593963291,"572,400,000.0","1,766,075,000.0",0.17387719337109045,0.09809525,0.09809525,"17,771,304,856.0","17,771,304,856.0",https://www1.hkexnews.hk/search/titlesearch.xhtml,https://www1.hkexnews.hk/search/titlesearch.xhtml
2,2019-03-31,0003.HK,HKD,2019-03-31,2019-03-31,Q1,"10,157,025,000.0","5,450,950,000.0","4,706,075,000.0",0.4633320288174933,"2,650,000.0",0.0,0.0,"2,996,375,000.0","-2,650,000.0","2,999,025,000.0","8,449,975,000.0","307,600,000.0","688,375,000.0","2,395,425,000.0",0.2358392344214965,"1,707,050,000.0",0.16806594450638845,0.0,"2,600,975,000.0",0.2560764593963291,"572,400,000.0","1,766,075,000.0",0.17387719337109045,0.09809525,0.09809525,"17,771,304,856.0","17,771,304,856.0",https://www1.hkexnews.hk/search/titlesearch.xhtml,https://www1.hkexnews.hk/search/titlesearch.xhtml
3,2019-01-31,0003.HK,HKD,2019-01-31,2019-01-31,Q1,"10,157,025,000.0","5,450,950,000.0","4,706,075,000.0",0.4633320288174933,"2,650,000.0",0.0,0.0,"2,996,375,000.0","-2,650,000.0","2,999,025,000.0","8,449,975,000.0","307,600,000.0","688,375,000.0","2,395,425,000.0",0.2358392344214965,"1,707,050,000.0",0.16806594450638845,0.0,"2,600,975,000.0",0.2560764593963291,"572,400,000.0","1,766,075,000.0",0.17387719337109045,0.09809525,0.09809525,"17,771,304,856.0","17,771,304,856.0",https://www1.hkexnews.hk/search/titlesearch.xhtml,https://www1.hkexnews.hk/search/titlesearch.xhtml
4,2018-09-30,0003.HK,HKD,2018-09-30,2018-09-30,Q3,"9,768,250,000.0","5,149,475,000.0","4,618,775,000.0",0.4728354618278607,"7,075,000.0",0.0,0.0,"2,572,975,000.0",0.0,"2,580,050,000.0","7,729,525,000.0","294,150,000.0","634,300,000.0","2,673,025,000.0",0.2736442044378471,"2,038,725,000.0",0.2087093389296957,0.0,"3,084,875,000.0",0.3158063112635324,"476,900,000.0","2,355,050,000.0",0.2410923143858931,0.1309525,0.1309525,"17,771,304,856.0","17,771,304,856.0",https://www1.hkexnews.hk/search/titlesearch.xhtml,https://www1.hkexnews.hk/search/titlesearch.xhtml


### Timeseries Quarterly Cash Flow Statement

Get quarterly cashflow statements.

In [ ]:
# #get cashflow statement df
# df_cashflow_statement = get_quarterly_cashflow_statement_timeseries(api_key, clean_symbols_list, 5000, 12)

In [ ]:
# #write df to csv
# df_cashflow_statement.to_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Dashboarding/Quarterly Cashflow Statement Timeseries.csv',index=True)

In [ ]:
#read from csv
df_cashflow_statement = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Dashboarding/Quarterly Cashflow Statement Timeseries.csv',
                                    index_col='Unnamed: 0')

In [ ]:
df_cashflow_statement.head()

,date,symbol,reportedCurrency,fillingDate,acceptedDate,period,netIncome,depreciationAndAmortization,deferredIncomeTax,stockBasedCompensation,changeInWorkingCapital,accountsReceivables,inventory,accountsPayables,otherWorkingCapital,otherNonCashItems,netCashProvidedByOperatingActivities,investmentsInPropertyPlantAndEquipment,acquisitionsNet,purchasesOfInvestments,salesMaturitiesOfInvestments,otherInvestingActivites,netCashUsedForInvestingActivites,debtRepayment,commonStockIssued,commonStockRepurchased,dividendsPaid,otherFinancingActivites,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,link,finalLink
0,2019-09-30,0003.HK,HKD,2019-09-30,2019-09-30,Q3,"1,766,075,000.0","688,375,000.0",0.0,0.0,"-900,000.0",0.0,"23,200,000.0",0.0,0.0,"252,700,000.0","2,478,100,000.0",0.0,nan,"-27,225,000.0","61,725,000.0",0.0,0.0,"-4,893,500,000.0",0.0,0.0,"-1,392,475,000.0","-375,000.0",0.0,"-11,250,000.0","-162,975,000.0","1,962,225,000.0","2,125,200,000.0","2,478,100,000.0","-1,763,225,000.0","714,875,000.0",https://www1.hkexnews.hk/search/titlesearch.xhtml,https://www1.hkexnews.hk/search/titlesearch.xhtml
1,2019-06-30,0003.HK,HKD,2019-06-30,2019-06-30,Q2,"1,766,075,000.0","688,375,000.0",0.0,0.0,"-900,000.0",0.0,"23,200,000.0",0.0,0.0,"252,700,000.0","2,478,100,000.0","-1,763,225,000.0",nan,"-27,225,000.0","61,725,000.0",0.0,0.0,"-4,893,500,000.0",0.0,0.0,"-1,392,475,000.0","-375,000.0",0.0,"-11,250,000.0","-162,975,000.0","1,962,225,000.0","2,125,200,000.0","2,478,100,000.0","-1,763,225,000.0","714,875,000.0",https://www1.hkexnews.hk/search/titlesearch.xhtml,https://www1.hkexnews.hk/search/titlesearch.xhtml
2,2019-03-31,0003.HK,HKD,2019-03-31,2019-03-31,Q1,"1,766,075,000.0","688,375,000.0",0.0,0.0,"-900,000.0",0.0,"23,200,000.0",0.0,0.0,"252,700,000.0","2,478,100,000.0",0.0,nan,"-27,225,000.0","61,725,000.0",0.0,0.0,"-4,893,500,000.0",0.0,0.0,"-1,392,475,000.0","-375,000.0",0.0,"-11,250,000.0","-162,975,000.0","1,962,225,000.0","2,125,200,000.0","2,478,100,000.0","-1,763,225,000.0","714,875,000.0",https://www1.hkexnews.hk/search/titlesearch.xhtml,https://www1.hkexnews.hk/search/titlesearch.xhtml
3,2019-01-31,0003.HK,HKD,2019-01-31,2019-01-31,Q1,"1,766,075,000.0","688,375,000.0",0.0,0.0,"-900,000.0",0.0,"23,200,000.0",0.0,0.0,"252,700,000.0","2,478,100,000.0",0.0,nan,"-27,225,000.0","61,725,000.0",0.0,0.0,"-4,893,500,000.0",0.0,0.0,"-1,392,475,000.0","-375,000.0",0.0,"-11,250,000.0","-162,975,000.0","1,962,225,000.0","2,125,200,000.0","2,478,100,000.0","-1,763,225,000.0","714,875,000.0",https://www1.hkexnews.hk/search/titlesearch.xhtml,https://www1.hkexnews.hk/search/titlesearch.xhtml
4,2018-09-30,0003.HK,HKD,2018-09-30,2018-09-30,Q3,"2,355,050,000.0","634,300,000.0",0.0,0.0,"23,375,000.0",0.0,"11,200,000.0",0.0,0.0,"219,325,000.0","2,359,725,000.0",0.0,nan,"-283,600,000.0","652,875,000.0",0.0,0.0,"-5,318,050,000.0",0.0,0.0,"-1,265,900,000.0","32,425,000.0",0.0,"70,225,000.0","-564,450,000.0","2,125,200,000.0","2,689,650,000.0","2,359,725,000.0","-1,686,500,000.0","673,225,000.0",https://www1.hkexnews.hk/search/titlesearch.xhtml,https://www1.hkexnews.hk/search/titlesearch.xhtml


### Timeseries Quarterly Company Key Metrics Statement

Get quarterly company key metrics statements.

In [ ]:
# #get key metrics statement
# df_key_metrics_statement = get_quarterly_key_metrics_statement_timeseries(api_key, clean_symbols_list, 5000, 12)

In [ ]:
# #write df to csv
# df_key_metrics_statement.to_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Dashboarding/Quarterly Key Metrics Statement Timeseries.csv',index=True)

In [ ]:
df_key_metrics = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Dashboarding/Quarterly Key Metrics Statement Timeseries.csv', 
                             index_col='Unnamed: 0')

In [ ]:
df_key_metrics.head()

,symbol,date,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,interestDebtPerShare,marketCap,enterpriseValue,peRatio,priceToSalesRatio,pocfratio,pfcfRatio,pbRatio,ptbRatio,evToSales,enterpriseValueOverEBITDA,evToOperatingCashFlow,evToFreeCashFlow,earningsYield,freeCashFlowYield,debtToEquity,debtToAssets,netDebtToEBITDA,currentRatio,interestCoverage,incomeQuality,dividendYield,payoutRatio,salesGeneralAndAdministrativeToRevenue,researchAndDdevelopementToRevenue,intangiblesToTotalAssets,capexToOperatingCashFlow,capexToRevenue,capexToDepreciation,stockBasedCompensationToRevenue,grahamNumber,roic,returnOnTangibleAssets,grahamNetNet,workingCapital,tangibleAssetValue,netCurrentAssetValue,investedCapital,averageReceivables,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
0,0003.HK,2019-06-30,Q2,0.5715407552963538,0.0993779024281232,0.13944389678078908,0.040226365244004116,0.4450207828903388,4.1212843172442835,3.6683237684704992,4.1212843172442835,2.166436303465999,"282,728,967,031.64624","313,709,767,031.64624",40.022219757321494,27.835804975536266,114.09102418451485,395.49427107067146,3.860276500078457,3.860276500078457,30.88598945376685,130.9620493364001,126.5928602686115,438.83163774316665,0.006246530090432229,0.0025284816320924877,0.3791367083284408,0.4583484941264869,12.933320809459698,0.7938096064511182,5.549577373211965,1.4031680421273165,0.004925123218252123,0.7884574550910918,0.0,0.00026090316800441067,0.05953162763317092,-0.7115229409628344,-0.17359659939795366,-2.5614309061194844,0.0,3.0356553315630674,0.015456795131478513,0.01388778579432638,-2.974483327382294,"-5,108,800,000.0","65,190,900,000.0","-42,308,300,000.0",0.5214703320289567,nan,"2,809,750,000.0","2,393,950,000.0",0.0,45.70377640594759,39.87543455727901,nan,1.9692027022145155,2.257028694463997,0.02411333331512849,-0.09921753153678498
1,0003.HK,2018-06-30,Q2,0.5496641962507337,0.1325198132091511,0.132782877741434,0.03788269941093852,0.7523420541337429,4.029141392835043,3.7010281762058583,4.029141392835043,2.320198225966441,"220,098,943,489.4242","249,240,043,489.4242",23.364572247874165,22.532075191505562,93.27313288176552,326.9322195245634,3.073874503889136,3.073874503889136,25.51532193478097,93.24269076773476,105.62249562530558,370.21804521434024,0.010699960493509414,0.003058737989954725,0.3158368841572502,0.4692500813509521,10.9019182386996,0.7617440173708363,6.930902600713923,1.0019850958578371,0.005751504209563945,0.5375257425532367,0.0,0.0007242853121081054,0.04322163112550432,-0.7147019250124484,-0.1726511913597625,-2.658836512691156,0.0,3.4660747189529215,0.019982288932494083,0.01824512718249311,-2.7423591230300595,"-7,834,500,000.0","65,772,100,000.0","-38,258,000,000.0",0.5717461953462909,nan,"2,767,850,000.0","2,387,200,000.0",0.0,46.9148796721996,41.97398763951665,nan,1.91836791714786,2.1441851265822787,0.032890335753619604,-0.09490017833049548
2,0003.HK,2017-06-30,Q2,0.4568589620154446,0.11727263519288575,0.11991589413281775,0.033470494921661736,0.5981108371711197,3.700900132799754,3.382122260601716,3.700900132799754,2.014550164920308,"195,564,105,323.066","222,313,705,323.066",23.45877830301277,24.086844989215702,91.76674267892919,328.7758673947228,2.9734093342516608,2.9734093342516608,27.381485729443256,92.68767485144663,104.31875807004177,373.74640494778464,0.010656991458412516,0.0030415857706472624,0.4089613963600979,0.4592121242518749,11.152520820087345,0.956062512722764,5.743734585090301,1.0225394350146944,0.005884515453891261,0.5521741738139507,0.0,0.00144412113374286,0.0465808863129385,-0.7208835812491202,-0.18921681831478146,-2.5876284318679468,0.0,3.124951519796808,0.01804718521607225,0.01797348983015036,-2.481320996276232,"-949,700,000.0","60,105,800,000.0","-35,184,600,000.0",0.5395630292986271,n

### Stock Insider Trading

Get stock insider trading information.

In [ ]:
# #get stock insider trading
# df_stock_insider_trading = get_stock_insider_trading(api_key, clean_symbols_list, 5000, 500)

In [ ]:
# #write df to csv
# df_stock_insider_trading.to_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Dashboarding/Stock Insider Trading.csv',index=True)

In [ ]:
#read from csv
df_stock_insider_trading = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Dashboarding/Stock Insider Trading.csv',
                                       index_col= 'Unnamed: 0')

In [ ]:
df_stock_insider_trading.head()

,symbol,transactionDate,reportingCik,transactionType,securitiesOwned,companyCik,reportingName,typeOfOwner,acquistionOrDisposition,formType,securitiesTransacted,price,securityName,link
0,A,2021-06-17,1614512,M-Exempt,"83,387.7768",1090872,Grau Dominique,officer: Senior Vice President,A,4,"1,968.0",40.8,Common Stock,https://www.sec.gov/Archives/edgar/data/000109...
1,A,2021-06-17,1614512,S-Sale,"81,419.7768",1090872,Grau Dominique,officer: Senior Vice President,D,4,"1,968.0",40.8,Common Stock,https://www.sec.gov/Archives/edgar/data/000109...
2,A,2021-06-17,1614512,M-Exempt,"18,066.0",1090872,Grau Dominique,officer: Senior Vice President,D,4,"1,968.0",40.8,Employee Stock Option (Right to Buy),https://www.sec.gov/Archives/edgar/data/000109...
3,A,2021-06-16,1614512,M-Exempt,"84,906.7768",1090872,Grau Dominique,officer: Senior Vice President,A,4,"3,487.0",145.011,Common Stock,https://www.sec.gov/Archives/edgar/data/000109...
4,A,2021-06-16,1614512,S-Sale,"81,419.7768",1090872,Grau Dominique,officer: Senior Vice President,D,4,"3,487.0",145.011,Common Stock,https://www.sec.gov/Archives/edgar/data/000109...


### Company News

Get company news information.

In [ ]:
# #get stock news
# df_stock_news = get_stock_news(api_key, clean_symbols_list, 5000, 25)

In [ ]:
# #write df to csv
# df_stock_news.to_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Dashboarding/Stock News.csv',index=True)

In [5]:
#read from csv
df_stock_news = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/FI Capstone/Data/For Dashboarding/Stock News.csv',index_col='Unnamed: 0')

In [6]:
df_stock_news.head()

,symbol,publishedDate,title,image,site,text,url
0,IIVIP,2020-08-19 18:02:40,II-VI (NASDAQ:IIVI) Shareholders Have Enjoyed ...,NaN,Yahoo,The II-VI Incorporated (NASDAQ:IIVI) share pri...,https://finance.yahoo.com/news/ii-vi-nasdaq-ii...
1,IIVIP,2020-08-13 12:13:35,"Dow Jones Today, Futures Backtrack As Cisco Sy...",NaN,Yahoo,"Tesla led the Nasdaq, Cisco Systems dragged on...",https://finance.yahoo.com/m/90fdd055-617a-3a9e...
2,IIVIP,2020-08-13 10:55:00,II-VI Incorporated Reports Fiscal 2020 Fourth ...,NaN,Yahoo,* Achieves Record Quarterly Revenues of $746.2...,https://finance.yahoo.com/news/ii-vi-incorpora...
3,IIVIP,2020-08-12 23:30:00,II-VI Incorporated to Acquire Ascatron and Out...,NaN,Yahoo,"* Ascatron AB - Kista, Sweden: Silicon carbide...",https://finance.yahoo.com/news/ii-vi-incorpora...
0,RZB,2021-05-27 06:04:03,IVOV: Midcap Value ETF Review,https://cdn.snapi.dev/images/v1/e/t/etf31-8430...,Seeking Alpha,IVOV strategy and facts. Comparing IVOV with m...,https://seekingalpha.com/article/4431584-ivov-...
